# OPENCV官方文档:
https://docs.opencv.org/3.0-beta/modules/highgui/doc/user_interface.html

关于为什么要有0xFF:
- On some systems, waitKey() may return a value that encodes more than just the ASCII keycode. 
- On all systems, we can ensure that we extract just the ASCII keycode by reading the last byte from the return value like this: 
```python
keycode = cv2.waitKey(1)
if keycode != -1: 
    keycode &= 0xFF
```

In [6]:
# 测试
# 人脸检测
import cv2

cap = cv2.VideoCapture(0)
cap.set(3,640) # set Width
cap.set(4,480) # set Height

faceCascade = cv2.CascadeClassifier('Cascades/haarcascade_frontalface_alt.xml')
eyeCascade = cv2.CascadeClassifier('Cascades/haarcascade_eye_tree_eyeglasses.xml')

while(True):
    ret, frame = cap.read()
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # gray 表示输入 grayscale 图像。
    # scaleFactor 表示每个图像缩减的比例大小。
    # minNeighbors 表示每个备选矩形框具备的邻近数量。数字越大，假正类越少。
    # minSize 表示人脸识别的最小矩形大小。
    faces = faceCascade.detectMultiScale(
        gray, 
        scaleFactor=1.1,
        minNeighbors=2,
        minSize=(10, 10),
#         flags=cv2.CASCADE_SCALE_IMAGE  # 还不知道什么用
    )
    for (x,y,w,h) in faces:
        # 画矩形框
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
        roi_color = frame[y:y+h, x:x+w]
        
    cv2.imshow('frame', frame)
    
    
    
    k = cv2.waitKey(10) & 0xff
    if k == ord('q'):  # press 'q' to quit  k == ord('q')
        break


cap.release()
cv2.destroyAllWindows()

        


# 1.采集人脸样本
## 人脸采集(拍照)，将人脸保存到 ./faces/name 路径下

In [1]:



import cv2
import os
import glob

cap = cv2.VideoCapture(0)
cap.set(3,640) # set Width
cap.set(4,480) # set Height

faceCascade = cv2.CascadeClassifier('Cascades/haarcascade_frontalface_alt.xml')
eyeCascade = cv2.CascadeClassifier('Cascades/haarcascade_eye_tree_eyeglasses.xml')

def get_face():
    name = input("input name:")
    path = "./faces/{}".format(name)
    if not os.path.exists(path):
            os.makedirs(path)
    while(True):
        ret, frame = cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        faces = faceCascade.detectMultiScale(
        gray, 
        scaleFactor=1.1,
        minNeighbors=2,
        minSize=(10, 10),
        # flags=cv2.CASCADE_SCALE_IMAGE  # 还不知道什么用
        )
        
        # 找最大的脸
        max_w = 0
        max_h = 0
        for (x,y,w,h) in faces:
            cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
            if max_w < w and max_h < h:
                max_w, max_h = w, h
                max_x, max_y = x, y
            
            # 截取人脸，并且都转化为200*200的固定大小
            face = cv2.resize(gray[max_y:max_y+max_h, max_x:max_x+max_w], (200,200))
            
            cv2.imshow('frame', frame)
            cv2.imshow('face', face)
        
        k = cv2.waitKey(10) & 0xff
        if k == ord('e'):  # press 'q' to quit  k == ord('q')
            num = len(glob.glob(pathname='{}/*.png'.format(path)))
            filename = "{}/{}.png".format(path,num)
            cv2.imwrite(filename, face)
            print("save face ", filename)
        if k == ord('q'):  # press 'q' to quit  k == ord('q')
            break
        
    cap.release()
    cv2.destroyAllWindows()

In [2]:
get_face()

input name:hit5heng
save face  ./faces/hit5heng/0.png
save face  ./faces/hit5heng/1.png
save face  ./faces/hit5heng/2.png
save face  ./faces/hit5heng/3.png
save face  ./faces/hit5heng/4.png
save face  ./faces/hit5heng/5.png


# 2.生成label

In [3]:
import imghdr
# 生成label文件
def get_label(path):
    fh = open("./faces/label.txt", 'w')
    # 表示人脸label
    label = 0
    for root, dirs, files in os.walk(path):
        # 循环每个文件夹
        for subdir in dirs:
            # 文件夹完整路径
            subdir_path = os.path.join(root,subdir)
            # 循环每个人物文件夹下面每张照片
            for file in os.listdir(subdir_path):
                # 照片完整路径
                filepath = os.path.join(subdir_path, file)
                # 判断文件类型是不是图片类型
                imgType = imghdr.what(filepath)
                if imgType == 'jpeg' or imgType == 'png' or imgType == 'jpg':
                    # 保存图片路径
                    fh.write(filepath);
                    fh.write(";")
                    # 标签
                    fh.write(str(label))
                    fh.write("\n")
            # 每个人的标签不一样，从0开始计数
            label = label + 1            
    fh.close()

In [4]:
get_label("faces")

# 3.训练自己的数据模型

In [5]:
import cv2
import numpy as np
# 保存图片数据
images = []
# 保存标签
labels = []
# 打开文件
fh = open("./faces/label.txt")
# 循环每一行
for line in fh:
    # 以;切分字符串
    arr = line.split(";")
    # 第0部分为图片路径，读取文件
    img = cv2.imread(arr[0],0)
    # 保存图片数据
    images.append(img)
    # 保存对应的标签数据
    labels.append(int(arr[1]))
    
# 安装opencv扩展包
# pip install opencv-contrib-python
# 定义人脸识别模型
model = cv2.face.EigenFaceRecognizer_create()
# model = cv2.face.FisherFaceRecognizer_create()
# model = cv2.face.LBPHFaceRecognizer_create()
# 训练模型
model.train(np.array(images), np.array(labels))
# 保存模型
model.save("./faces/predict_face.xml")

# 4.人脸识别 （摄像头实时识别人脸并且写上名字）

In [25]:
import os
# 获取根目录路径、子目录路径，根目录和子目录下所有文件名
# for root, subDir, files in os.walk("./faces"):
#     print(root, "\n...........\n", subDir, "\n.......\n", files)
for _, subDir, _ in os.walk("./faces"):
    print(subDir)
    if len(subDir) != 0:
        name = subDir

print(name)
# for lists in os.listdir("./faces"):
#     print(lists)

['wuheng']
[]
['wuheng']


In [6]:
import cv2
import os
import glob

cap = cv2.VideoCapture(0)
cap.set(3,640) # set Width
cap.set(4,480) # set Height

faceCascade = cv2.CascadeClassifier('Cascades/haarcascade_frontalface_alt.xml')
eyeCascade = cv2.CascadeClassifier('Cascades/haarcascade_eye_tree_eyeglasses.xml')

# 定义人物名字
for _, subDir, _ in os.walk("./faces"):
    if len(subDir) != 0:
        name = subDir
        
# 定义人脸识别模型
model = cv2.face.EigenFaceRecognizer_create()
# 载入训练好的模型
model.read('./faces/predict_face.xml')

while(True):
    ret, frame = cap.read()
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # gray 表示输入 grayscale 图像。
    # scaleFactor 表示每个图像缩减的比例大小。
    # minNeighbors 表示每个备选矩形框具备的邻近数量。数字越大，假正类越少。
    # minSize 表示人脸识别的最小矩形大小。
    faces = faceCascade.detectMultiScale(
        gray, 
        scaleFactor=1.1,
        minNeighbors=2,
        minSize=(10, 10),
#         flags=cv2.CASCADE_SCALE_IMAGE  # 还不知道什么用
    )
    for (x,y,w,h) in faces:
        # 画矩形框
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
        
        # 预测人物
        face = cv2.resize(gray[y:y+h,x:x+w], (200,200))
        params = model.predict(face)
        # 写上人物名字
        cv2.putText(frame,name[params[0]],(x,y-50),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
        
    cv2.imshow('frame', frame)
    
    
    
    k = cv2.waitKey(10) & 0xff
    if k == ord('q'):  # press 'q' to quit  k == ord('q')
        break


cap.release()
cv2.destroyAllWindows()


# 读取树梅派摄像头: MJPG-streamer方式
- 参考https://blog.csdn.net/m0_38106923/article/details/86562451#

## 启动数梅派
- 进入文件夹
`cd ./share/mjpg-streamer-master/mjpg-streamer-experimental`
- 指令启动普通USB摄像头指令：
`./mjpg_streamer -i "./input_uvc.so" -o "./output_http.so -w ./www"  `
- 启动树莓派专用摄像头RaspiCamera的指令：
`./mjpg_streamer -i "./input_raspicam.so" -o "./output_http.so -w ./www"`

## 进入页面
- 主页 http://192.168.1.101:8080/
- 静态 http://192.168.1.101:8080/?action=snapshot
- 动态 http://192.168.1.101:8080/?action=stream
- 或者 http://192.168.1.101:8080/javascript_simple.html

## opencv-python读取stream
- 参考1:读取stream https://blog.csdn.net/xiao__run/article/details/76342634
- 参考2:读取pdf中的图片 https://blog.csdn.net/u014267567/article/details/73252401

In [4]:
# 检测人脸

import cv2
import urllib
import numpy as np
import sys


def detect(img, cascade):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # 不需要转换为灰度图来检测
    faces = cascade.detectMultiScale(
        gray, 
        scaleFactor=1.1,
        minNeighbors=2,
        minSize=(10, 10),
        # flags=cv2.CASCADE_SCALE_IMAGE  # 还不知道什么用
    )
    for (x,y,w,h) in faces:
        # 画矩形框
        cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
        # roi_color = frame[y:y+h, x:x+w]  # 截取人脸
    return img


def get_steam():
    faceCascade = cv2.CascadeClassifier('Cascades/haarcascade_frontalface_alt.xml')

    host = "192.168.1.101"
#     if len(sys.argv)>1:
#         host = sys.argv[1]
    hoststr = 'http://' + host + '/?action=stream'
    print('Streaming ' + hoststr)
    print('Print "q" to quit')

    # stream=urllib.request.urlopen(hoststr)
    stream = urllib.request.urlopen("http://192.168.1.101:8080/?action=stream")

    bytes1 = b''
    while True:
        bytes1 += stream.read(1024)
        startmark = bytes1.find(b'\xff\xd8')
        endmark = bytes1.find(b'\xff\xd9')
        if startmark != -1 and endmark != -1:
            jpg = bytes1[startmark:endmark + 2]
            bytes1 = bytes1[endmark + 2:] #flags = 1 for color image 
            i = cv2.imdecode(np.fromstring(jpg, dtype=np.uint8),flags=1) # print i.shape

            img = detect(i, faceCascade)

            cv2.imshow("pi stream",img)
            if cv2.waitKey(1) & 0xFF == ord('q'):
    #             exit(0)
                break

    stream.close()
    cv2.destroyAllWindows()

In [5]:
get_steam()

Streaming http://192.168.1.101/?action=stream
Print "q" to quit


URLError: <urlopen error [Errno 111] Connection refused>